In [71]:
%%html
<link rel="stylesheet" href="static/hyrule.css" type="text/css">

# Handling Data with Pandas

## Objectives

* Review different ways to pull data into pandas and the link between objects in python and pandas
* Understand the differences between a DataFrame and a Series
* Practice part of the ACES data exploration model
* Learn imputation strategies


## Code Dictionary

Each class we'll make sure there's one location that shows any new functionality introduced, with explanations of how each method works.


`pd.read_csv`: Reads in a csv file (by default) as a DataFrame. Does much more! check out [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.parsers.read_csv.html)

`pd.DataFrame`: Pandas' primary object, a 2-dimensional array (matrix).

`pd.Series`: Pandas' other object, a 1-dimensional array (vector).

functions | description
----------|------------
`.head()` | returns back the head of the pandas object (works for both DataFrame and Series)
`.describe()` | returns statistical results back for the pandas object. 
`.isnull()` | returns back true for each value if it is null. Opposite of `.notnull()`
`.dropna()` | returns back the object with the NA values dropped. `.dropna()` and the following four functions **do not** update the original pandas object. <br /> Can use the `inplace` argument to change that.
`.ffill()` | front fills the missing data.
`.bfill()` | back fills the missing data.
`.fillna()` | fills missing data with whatever pass in as an argument. 
`.apply()` | apply a function to either a series (column) or DataFrame (by row)

parameters | description
-----------|------------
`.columns` | returns back the columns
`.index` | returns back the index of the object
`[statement]` | commonly used for filtering to either a set of columns, rows, or data that is true to the boolean statement

## Lecture Notes

### A quick introduction to iPython Notebook:

Consider iPython notebook to be a great organization tool, but it takes much **practice** to keep it organized. Early recommendations while we practice using this tool in class:

1. It is very easy to want to edit previous python cells. Until you're really confortatable with Notebook, please don't! Everything that you run stays **in the notebook**, so the moment you accidentally delete code that you might have been using, you lose your work the next time you run the notebook.
2. The `mode` of Notebook is very similar to vim, so vim users should feel comfortable:
    1. The natural mode is a command mode. If you press `esc`, you should be in this mode. It's for gettinng around, changing cell types, and other commands. While in command mode, if you press `h`, it'll let you know anything else you can do.
    2. The other primary mode is edit mode. Pressing `return` on a cell will put you into edit mode (this would be similar to pressing `i` or `a` in vim). It's for editing cells, be they headers, markdown, code. We won't need nbconvert for the purposes of this course.
3. Most developers initialize a notebook with customized pandas settings. Examples of those defaults are included below:


In [1]:
import pandas as pd
import numpy as np
from __future__ import division
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.mpl_style = 'default'

They are customizable to your taste, and we will add more later. 

### Exploratory Data Analysis

Primarily our goals for exploring data are the following:

* **A**ssemble.
* **C**lean.
* **E**xplore.
* **S**ummarize.

Today we will review some common pandas commands (provided above) to focus on for this data exploration model.

### Reading in Data

Pandas will work across a variety of data inputs, including csv, excel, JSON, and using additional python libraries to connect to databases. For today, we'll focus on using the csv input. We'll use data about heart disease from the UC Irvine Machine Learning data repository[1]. 

Given this directory: http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/

There are a few different data types, some processed, some not, and a "names" file. The names file will expose for us the columns for the processed data:
```
7. Attribute Information:
-- Only 14 used
  -- 1. #3  (age)       
  -- 2. #4  (sex)       
  -- 3. #9  (cp)        
  -- 4. #10 (trestbps)  
  -- 5. #12 (chol)      
  -- 6. #16 (fbs)       
  -- 7. #19 (restecg)   
  -- 8. #32 (thalach)   
  -- 9. #38 (exang)     
  -- 10. #40 (oldpeak)   
  -- 11. #41 (slope)     
  -- 12. #44 (ca)        
  -- 13. #51 (thal)      
  -- 14. #58 (num)       (the predicted attribute)
```

Let's grab those fields as headers, and the processed cleveland data to work with in pandas (the .names file refers that this is the primary file used in research). Pandas io tools [2] handles http requests to grab files from the internet, though reminder that when doing so, it only saves the file in memory (in python), and not as a file on your machine.

In [2]:
header_row = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num',]
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
heart_data = pd.read_csv(url, header=0, na_values='?')
heart_data.columns = header_row

In [3]:
print heart_data.head()

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  ca  thal  num
0   67    1   4       160   286    0        2      108      1      1.5      2   3     3    2
1   67    1   4       120   229    0        2      129      1      2.6      2   2     7    1
2   37    1   3       130   250    0        0      187      0      3.5      3   0     3    0
3   41    0   2       130   204    0        2      172      0      1.4      1   0     3    0
4   56    1   2       120   236    0        0      178      0      0.8      1   0     3    0


### Pandas is really just python
Pandas is a library for python, built heavily around the task of manipulating and presenting data. If you're writing pandas code, you're writing python code! Pandas contains (primarily) two new python objects:

* **DataFrame**: a wrapper around a 2 dimensional numpy ndarray (in math, we call this a matrix)
* **Series**: a wrapper around a 1 dimensional numpy ndarray (in math, we call this a vector)

### Math Jargon

When working through data matrices and vectors, we'll also often use the words feature and observation. 

* **Feature**: a feature is represented by a column. It is a segmentation of your data. Features are usually either continuous values (representing -inf to inf, and 1 < 2) or categorical values (each value represents its own space; 1 !< 2).
* **Observation**: an observation is a row of your data. It should represent a single entity (for example, a survey responder).
* **Target Variable**: often we'll be working with a column called a target variable, or predicted value. In data analysis, it is often the goal to be able to statistically explain this variable using the observations and features.

### DataFrames behave like lists
DataFrames support many of the functionalities of lists, like slicing.

In [4]:
## Lists
import random
random_list = [random.random() for i in xrange(300)]

#print random_list[3:14]
#print random_list[280:]
#print random_list[:20]

#print len(random_list)

## DataFrame
#print heart_data[3:14]
#print heart_data[280:]
#print heart_data[:20]

#print len(heart_data)

### DataFrames also behave like dictionaries
DataFrames support returning by column in a similar way a dictionary returns by key. Note that passing in a string for a key will return a pandas **Series**, while a list of keys will return a **DataFrame**.

The following table of code shows similar code, dependent on your object type:

action | dictionary | DataFrame
-------|-----------|----------
return values for a key | `some_dict['a']` | `heart_data['age']`
creating a new key | `some_dict['d'] = [i**2 for i in some_dictionary['a']]` | `heart_data['random_var'] = random.random()`
filtering results for a key | `[i for i in some_dict['a'] if i > .5]` | `heart_data[heart_data['random_var'] >.7]`


In [5]:
## Dictionary
some_dict = {k: [random.random() for i in xrange(20)] for k in ['a', 'b', 'c',]}
some_dict['d'] = [i**2 for i in some_dict['a']]

#print some_dict['a']
#print some_dict['b']
my_list = [i for i in some_dict['a'] if i > .5]
new_list = [i for i in some_dict['a'] if i < .5]
#print len(my_list), len(new_list)

## DataFrame
heart_data['random_var'] = [random.random() for i in heart_data.index]

#print heart_data['age'].head()
#print heart_data['sex'].head()
#print heart_data[['cp', 'oldpeak']].head()

# Note on Printing Types -- What do we get back?
print type(heart_data['sex'])
print type(heart_data[['sex']])
print type(heart_data[['cp', 'oldpeak']])

# Creating a subset:
# Since random.random uses a flat distribution, consider the below as one approach to pick a random subset.
heart_subset = heart_data[heart_data['random_var'] >.7].head()

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


### Practice: Selecting and subsetting data

Let's write code to do the following things:

1. Return back the `head()` of the `chol` column
2. Return back all rows where `thalach` is below 150.
3. Return back all rows where `age` is above the median (hint, you can get the median of a column using `df.col.median()`)
4. Summarise the resting heart beat rate when the pain type is asymptomatic (look up columns in the text file, and use `.describe()`)


### Cleaning Data

Our primary task for today to clean is to find and handle missing values. Data can be missing for different reasons:

* There was no response value. This is common in True/False data, where True could be a yes, False, could be a no, and NA just means there was no answer.
* The data was poorly handled. Missing data happens all the time on poor data imports.
* The missing data really should have just been a 0.

To find missing data, we can use pandas `.describe()` function, which uses `count` as a "count of non null values" field, and the `.isnull()` function once we've identified rows missing data.


In [6]:
heart_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,random_var
count,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,302.000000,298.000000,300.000000,302.000000,302.000000
mean,54.410596,0.678808,3.165563,131.645695,246.738411,0.145695,0.986755,149.605960,0.327815,1.035430,1.596026,0.674497,4.730000,0.940397,0.480928
std,9.040163,0.467709,0.953612,17.612202,51.856829,0.353386,0.994916,22.912959,0.470196,1.160723,0.611939,0.938202,1.941563,1.229384,0.291030
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.000871
25%,48.000000,0.000000,3.000000,120.000000,211.000000,0.000000,0.000000,133.250000,0.000000,0.000000,1.000000,0.000000,3.000000,0.000000,0.223201
50%,55.500000,1.000000,3.000000,130.000000,241.500000,0.000000,0.500000,153.000000,0.000000,0.800000,2.000000,0.000000,3.000000,0.000000,0.466141
75%,61.000000,1.000000,4.000000,140.000000,275.000000,0.000000,2.000000,166.000000,1.000000,1.600000,2.000000,1.000000,7.000000,2.000000,0.732741
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000,4.000000,0.999723


In [7]:
## Looks like we have 4 missing values in the ca column, or the number of primary vessels, so let's focus on that one first.
print heart_data[heart_data['ca'].isnull()]

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  ca  thal  num  random_var
165   52    1   3       138   223    0        0      169      0      0.0      1 NaN     3    0    0.132960
191   43    1   4       132   247    1        2      143      1      0.1      2 NaN     7    1    0.718346
286   58    1   2       125   220    0        0      144      0      0.4      2 NaN     7    0    0.399588
301   38    1   3       138   175    0        0      173      0      0.0      1 NaN     3    0    0.451119


We can choose the handle the data a few different ways. This handling is called data imputation.

1. Remove the data. This makes sense if we think it's bad data.
2. Fill the data. Common techniques would be to fill with some default value (0), or backfill/forwardfill the data, based on the sort.
3. interpolate the data. This technique is an estimation, sometimes with machine learning techniques. More on this later!

In [8]:
""" Trial 1: Drop missing values. 
Pandas defines this on the axis (think axis of a matrix):
0: along the columns
1: along the rows
"""
#print heart_data[285:290]

dropped_rows_heart_data = heart_data.dropna()
dropped_ca_thal_heart_data = heart_data.dropna(1)

#print dropped_rows_heart_data[285:290]
#print dropped_ca_thal_heart_data[285:290]

""" Trial 2: Fill the data
.ffill(): fills forward
.bfill(): fills backward
.fillna(): fills based on some value
"""
#print heart_data[285:290]['ca'].ffill()
#print heart_data[285:290]['ca'].bfill()
#print heart_data[285:290]['ca'].fillna(0) # fill with 0s
print heart_data['ca'][285:290].fillna(int(heart_data['ca'].mode())) # fill with the most common value

285    2
286    0
287    0
288    0
289    0
Name: ca, dtype: float64


### Practice Together: Filling Missing Data

One other column was missing data in the heart disease data set.

1. You can find it if you look at your counts in .describe. What was it?
2. Look up the information about that column in the names file. What is that column about?
3. Determine three different ways to fill that column. Which ways seems most ideal for this case?

### Constructing new data

One commont technique to data manipulation is to generate new data based on data already in the DataFrame. Pandas uses a function called `.apply()` in order to run such functions, either named functions (`def`) or nameless functions (`lambda`). `apply()` is particularly helpful for iterating through pandas data.

Try it out by making a function where the patients resting heart beat (trestbps) was higher than their maximum heart rate achieved (thalach).

In [9]:
def high_resting(row):
    if row['trestbps'] > row['thalach']:
        return 1
    else:
        return 0

# axis = 1 means what again?
heart_data['high_resting'] = heart_data.apply(high_resting, axis=1)

# This would work the same way!
heart_data['high_resting'] = heart_data.apply(lambda row: 1 if row['trestbps'] > row['thalach'] else 0, axis=1)

# axis = 0 means what again?
heart_data['resting2x'] = heart_data['trestbps'].apply(lambda x: x*2)

## On Your Own

You're going work on similar questions with another data set from UC Irvine on vehicle mileage per gallon. You can find it with this link:

http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/

1. Bring the data into a DataFrame with pandas. The file is `auto-mpg.data`. Since it is space seperated, you'll have to tell read_csv to use spaces (`'\s+'`) and not commas (`','`) as the delimeter. Likewise, you have to name the columns.
2. Compare the data for cars from the year 1970 and the year 1982. In general in this data set, have cars changed in terms of mpg, horsepower, etc? (any of the continuous values)
3. Horsepower is missing several values. What are some basic techniques to fill in the missing data?

** more advanced **

4. Consider splitting the data by year and filling in horsepower that way. What would the python code look like to hande this?
5. A handy way to work with strings is to `.split()` them by a value into a python list. Write an `apply()` function that generates a new column called "manufacturer," using the last column's data.

## Review, Next Steps, Reading

For next class:

0. On our off day, please review this whole notebook again, and make sure you can follow along without guidance! Use slack when you have questions. You should do this with each class notebook.
1. Read through the documentation for [split, apply, combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html). It's a technique we will be using in more detail with the next few classes.
2. To understand the choices in data storage, read about [tidy data](http://vita.had.co.nz/papers/tidy-data.pdf). 
3. Additional resource for [tidy data](http://www.prometheusresearch.com/good-data-management-practices-for-data-analysis-tidy-data-part-2/).
4. The two visualisation packages we will be using are [matplotlib](http://matplotlib.org/) and [seaborn](http://stanford.edu/~mwaskom/software/seaborn/). Consider trying out some sample code. You'll need to install seaborn with `conda`. We'll do that as a class on Wednesday.
5. Questions to think about when doing [Exploratory Data Analysis](http://www.itl.nist.gov/div898/handbook/eda/section3/eda32.htm)

###### Resource links from today
1. [UC Irvine Data Repository](http://archive.ics.uci.edu/ml/)
2. [Pandas IO Cookbook](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#data-in-out)